In [ ]:
!pip -q install langchain openai tiktoken "pinecone-client[grpc]" apache_beam mwparserfromhell cohere python-dotenv

In [ ]:
# Exemplo de como carregar chaves de API no Google Colab.
from google.colab import files
from dotenv import load_dotenv

files_uploaded = files.upload()
load_dotenv()

In [ ]:
# Exemplo de uso do WebBaseLoader para carregar documentos a partir de URLs.
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
    "https://www.icmc.usp.br/noticias",
    "https://www.icmc.usp.br/graduacao/engenharia-de-computacao"]
)
docs = loader.load()

In [ ]:
# Exemplo de uso do divisor de texto CharacterTextSplitter do LangChain.
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

texts = text_splitter.split_documents(docs)

In [ ]:
# Exemplo de uso do divisor de texto RecursiveCharacterTextSplitter do LangChain.
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

texts = text_splitter.split_documents(docs)

In [ ]:
# Exemplo de uso do modelo text-embedding-ada-002 da OpenAI por meio do LangChain.
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

embeddings = embeddings_model.embed_documents([
    "Podemos destacar ao menos três frentes de pesquisa feita \
    pelos CEPIDs : da legitimidade e da impunidade e dos padrões \
    urbanos e criminais."
])

embedded_query = embeddings_model.embed_query("Quais são as frentes \
                de pesquisa do projeto feito pelos CEPIDs?")

In [ ]:
# Exemplo de uso do modelo de código aberto multilingual-e5-base por meio do LangChain.
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base"
)

embeddings = embeddings_model.embed_documents([
    "Podemos destacar ao menos três frentes de pesquisa feita \
    pelos CEPIDs : da legitimidade e da impunidade e dos padrões \
    urbanos e criminais."
])

embedded_query = embeddings_model.embed_query("Quais são as frentes \
                de pesquisa do projeto feito pelos CEPIDs?")

In [ ]:
# Função utilizada para gerar embeddings e inserir os vetores em um índice do Pinecone.
from uuid import uuid4

def insert_index(index, texts, metadatas, embeddings_model):
    # Gera ids unicos
    ids = [str(uuid4()) for _ in range(len(texts))]
    # Realiza embedding dos textos
    embeds = embeddings_model.embed_documents(texts)
    # Insere no índice
    index.upsert(vectors=zip(ids, embeds, metadatas))

In [ ]:
# Procedimento para inserção de chunks no Pinecone.
import pinecone
import datetime

# Carrega o índice
index = pinecone.GRPCIndex(index_name)

batch_limit = 100
texts = []
metadatas = []

for chunk in chunks:
    # Obter texto e metadados do chunk
    chunk_text = chunk.page_content
    chunk_metadatas = chunk.metadata
    chunk_metadatas["year"] = datetime.datetime.now().year
    chunk_metadatas["text"] = chunk_text

    # Adiciona às listas de textos e metadados
    texts.append(chunk_text)
    metadatas.append(chunk_metadatas)

    # Se ultrapassou o tamanho limite do lote (batch)
    if len(texts) >= batch_limit:
        insert_index(index, texts, metadatas, embeddings_model)
        texts = []
        metadatas = []

# Se ainda restam dados a serem inseridos
if len(texts) > 0:
    insert_index(index, texts, metadatas, embeddings_model)

In [ ]:
# Exemplo de realização de consulta no Pinecone por meio do LangChain.
from langchain.vectorstores import Pinecone

vectorstore = Pinecone.from_existing_index(index_name,
                                           embeddings_model)

query = "O que significa a sigla CEPIDs?"
results = vectorstore.similarity_search(
    query,
    k=3,
    filter={
        "language": {"\$eq": "pt-BR"},
        "year": {"\$gte": 2022}
    }
)

In [ ]:
# Exemplo de utilização do modelo gpt-3.5-turbo por meio do LangChain.
from langchain.chat_models import ChatOpenAI

openai = ChatOpenAI(model_name="gpt-3.5-turbo")
result = openai.predict("O que são Large Language Models?")

In [ ]:
# Exemplo de construção de prompt por meio do LangChain.
from langchain.prompts import ChatPromptTemplate

template = """
    Use os seguintes trechos de contexto para responder à pergunta no
    final. Se você não sabe a resposta, apenas diga que não sabe, não
    tente inventar uma resposta.
    Contexto: {contexto}
    Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)
prompt = prompt_template.format(
    contexto="Bruno nasceu em Itapeva",
    pergunta="O que você sabe sobre Itapeva?")
resposta = openai.predict(prompt)

In [ ]:
# Chain completa para QA com RAG por meio do LangChain.

# Carrega modelo de embedding
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Conecta ao Pinecone
index_name = 'noticias-icmc'
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

# Carrega o index do Pinecone (vector database)
vectorstore = Pinecone.from_existing_index(index_name,
                                           embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Carrega LLM
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

# Define template de prompt
template = """
    Use os seguintes trechos de contexto para responder à pergunta no
    final. Se você não sabe a resposta, apenas diga que não sabe, não
    tente inventar uma resposta.
    Contexto: {contexto}
    Pergunta: {pergunta}
"""

prompt_template =  ChatPromptTemplate.from_template(template)

# Define chain
chain = {
    "contexto": itemgetter("pergunta") | retriever,
    "pergunta": itemgetter("pergunta")
} | prompt_template | llm | StrOutputParser()